<a href="https://colab.research.google.com/github/pedrogengo/DLforNLP/blob/main/Aula_4_Exerc%C3%ADcio_Pedro_Gengo_NN_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook de referência 

Nome: Pedro Gabriel Gengo Lourenço

## Instruções

- Treinar uma rede neural como classificador binário na tarefa de análise de sentimentos usando dataset IMDB.

- Experimentar e reportar a acurácia usando 3 diferentes tipos de features como entrada:
    1) Bag-of-words booleano
    2) Bag-of-words com contagem das palavras (histograma das palavras)
    3) TF-IDF

Deve-se implementar o laço de treinamento e validação da rede neural.

Neste exercício usaremos o IMDB com 20l exemplos para treino, 5k para desenvolvimento e 25k para teste.

## Preparando Dados

Primeiro, fazemos download do dataset:

In [ ]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzf aclImdb.tgz

File ‘aclImdb.tgz’ already there; not retrieving.



## Carregando o dataset

Criaremos uma divisão de treino (80%) e validação (20%) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [ ]:
import os
import random


def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [True] * len(x_train_pos) + [False] * len(x_train_neg)
y_test = [True] * len(x_test_pos) + [False] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

n_train = int(0.8 * len(x_train))

x_valid = x_train[n_train:]
y_valid = y_train[n_train:]
x_train = x_train[:n_train]
y_train = y_train[:n_train]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print('3 últimas amostras treino:')
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print('3 primeiras amostras validação:')
for x, y in zip(x_valid[:3], y_test[:3]):
    print(y, x[:100])

print('3 últimas amostras validação:')
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])

20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.
3 primeiras amostras treino:
False I go to UCSB and take some classes with the executive producer, Alison Anders. She's a superb teache
True The barbarians maybe´s not the best film that anybody of us have seen, but really????........It´s so
True "Der Todesking" is not exactly the type of film that makes you merry Jörg Buttgereit's second cult 
3 últimas amostras treino:
False This is a really dumb movie. It could be fun with the cool looking aliens and the country setting, b
True Alex Winter and Keanu Reeves return as the two dopes from San Dimas who get sent on another trip of 
False In an attempt to bring back the teen slasher genre that was taken away by spoofs like Scary Movie an
3 primeiras amostras validação:
True Spoilers.<br /><br />First off, nothing really happened in this movie, other than a woman bleeding i
True Just a regular Jason lee movie, There were some parts of the movie that were 

## Processamento dos textos

Nessa etapa, como descrito no enunciado do exercício, iremos realizar três tipos de vetorização:

1. BoW booleano
2. BoW com base na frequência
3. TF-IDF

É muito importante ressaltar a importância de aplicar o "fit" apenas no treino, ou seja, utilizar apenas o vocabulário do treino, e utilizar o que foi encontrado para o teste/validação.

Para isso, usarei a estrutura de fit/transform bastante conhecida da biblioteca sklearn. Começarei, então, criando uma classe abstrata que será herdada na criação das outras.

In [ ]:
from abc import ABC, abstractmethod
 
class Transformer(ABC):
 
  @abstractmethod
  def fit(self):
      pass
    
  @abstractmethod
  def transform(self):
      pass

## BoW (booleano e com frequência)

In [ ]:
from collections import Counter
import numpy as np
import torch

class BagOfWords(Transformer):
  '''
  Essa classe realiza a transformacao de uma lista de palavras
  para uma lista de inteiros.

  Attrs:
    boolean(bool): Flag que define se o vetor gerado sera com base na
      frequencia (contagem) ou com base na ocorrencia ou nao (bool) de
      uma palavra do vocabulario.
    max_size(int): Define o tamanho maximo do vocabulario. Caso usado com
      use_unknown = True, o vocabulario tera o tamanho de max_size + 1.
    stopwords(list): Define a lista de palavras que serao desconsideras na
      geracao do vocabulario.
    use_unknown(bool): Flag que define o uso ou nao de um elemento para
      palavras que nao existem no vocabulario.
  '''

  def __init__(self, boolean=False, max_size=None, stopwords = [], use_unknown=False):
    self.max_size = max_size
    self.boolean = boolean
    self.stopwords = stopwords
    self.use_unknown = use_unknown

  def _create_vocab(self, tokenized_texts):
    '''
    Cria o vocabulario que sera utilizado na transformacao do vetores
    de palavras para vetores de inteiros.

    Args:
      tokenized_texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    
    Return:
      vocab(dict): Dicionario onde as chaves sao as palavras do vocabulario
        e os valores representam o indice da palavra no vetor a ser gerado.
    '''
    counter = Counter()
    for text in tokenized_texts:
      counter.update(text)
    for stop_word in self.stopwords:
      if stop_word in counter.keys():
        del counter[stop_word]
    vocab = {element[0]: index for index, element in enumerate(counter.most_common(self.max_size))}
    if self.use_unknown:
      vocab['unknown'] = len(vocab)
    return vocab
  
  def fit(self, texts):
    '''
    Metodo que cria os argumentos que serao utilizados nas
    transformacoes posteriores. Esse metodo so deve ser utilizado 
    sobre o conjunto de treino.

    Args:
      texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    '''
    vocab = self._create_vocab(texts)
    self.vocabulary = vocab

  def transform(self, texts):
    '''
    Realiza a transformacao de uma lista de tokens para uma
    lista de inteiros com base no vocabulario criado na etapa
    de fit.

    Args:
      texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    
    Return:
      bow_texts(torch.tensor): Array contendo os vetores de tokens
        transformados para vetores de inteiros de tamanho fixo.
    '''
    transformed_texts = []
    if self.use_unknown:
      unknown = self.vocabulary.get('unknown')

    for i, text in enumerate(texts):
      bow_text = torch.zeros(len(self.vocabulary))
      counter = Counter(text)

      if self.use_unknown:
        index = [self.vocabulary.get(key, unknown) for key in counter.keys() if key not in self.stopwords]
      else:
        index = [self.vocabulary[key] for key in counter.keys() if key in self.vocabulary.keys()]

      if self.boolean:
        bow_text[index] = 1
      else:
        values = [value for key, value in counter.items() if (self.use_unknown and key not in self.stopwords)
                    or key in self.vocabulary.keys()]
        bow_text[index] = torch.Tensor(values)

      transformed_texts.append(bow_text)

    return torch.vstack(transformed_texts).float()

In [ ]:
## Vocabulario deve ser: {'a': 0, 'texttinho': 1, 'testando': 2, 'b': 3, 'c': 4, 'unknown': 5}
texts_test = [['text', 'texttinho', 'texttinho', 'testando'], ['a', 'b', 'a', 'a', 'c']]

## Testando BoW com frequencia
bow = BagOfWords(boolean=False, stopwords=['text'], use_unknown=True)
bow.fit(texts_test)
assert torch.all(bow.transform(texts_test) == torch.Tensor([[0., 2., 1., 0., 0., 0.], [3., 0., 0., 1., 1., 0.]]))

## Testando BoW booleano
bow = BagOfWords(boolean=True, stopwords=['text'], use_unknown=False)
bow.fit(texts_test)
assert torch.all(bow.transform(texts_test) == torch.Tensor([[0., 1., 1., 0., 0.], [1., 0., 0., 1., 1.]]))

## TF-IDF

$$\text{TF-IDF}(t, d, C) = tf(t, d) * idf(t, C)$$

Abrindo as funções definidas na equação principal:
- $tf(t, d) = \text{numero de vezes que o termo t aparece no documento d}$
- $idf(t, C) = \log{\frac{C}{n_t}}$ 

Onde: 

- $\text{t: token ou termo;}$
- $\text{d: documento(frase, enunciado, etc);}$
- $\text{C: Corpus (conjunto de documentos).}$
- $n_t\text{: numero de documentos onde o token t aparece.}$


 



In [ ]:
from collections import Counter
import numpy as np

class TfIdf(Transformer):
  '''
  Essa classe realiza a transformacao de uma lista de palavras
  para uma lista de inteiros utilizando TFIDF.

  Attrs:
    max_size(int): Define o tamanho maximo do vocabulario. Caso usado com
      use_unknown = True, o vocabulario tera o tamanho de max_size + 1.
    stopwords(list): Define a lista de palavras que serao desconsideras na
      geracao do vocabulario.
  '''

  def __init__(self, max_size=None, stopwords = []):
    self.max_size = max_size
    self.stopwords = stopwords

  def _count_tokens_in_doc(self, tokenized_texts):
    '''
    Realiza a contagem de em quantos documentos uma mesma
    palavra aparece, desconsiderando as stopwords.

    Args:
      tokenized_texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    
    Return:
      counter(collections.Counter): Objeto da classe Counter com todos
        os elementos do conjunto de treino.
    '''
    counter = Counter()
    for text in tokenized_texts:
      counter.update(set(text))
    for stop_word in self.stopwords:
      if stop_word in counter.keys():
        del counter[stop_word]
    return counter
  
  def _create_idf(self, counter):
    '''
    Cria o vetor de idf para cada um dos tokens do conjunto de treino.

    Args:
      counter(collections.Counter): Objeto da classe Counter com todos
        os elementos do conjunto de treino.
    
    Return:
      idf(np.array): Array contendo o valor de idf para cada um dos tokens
        do conjunto de treino.
    '''
    idf = [self.len_corpus/count for token, count in counter.most_common(self.max_size)]
    return np.log(idf)

  def _create_vocab(self, counter):
    '''
    Cria o vocabulario que sera utilizado na transformacao do vetores
    de palavras para vetores de inteiros.

    Args:
      counter(collections.Counter): Objeto da classe Counter com todos
        os elementos do conjunto de treino.
    
    Return:
      vocab(dict): Dicionario onde as chaves sao as palavras do vocabulario
        e os valores representam o indice da palavra no vetor a ser gerado.
    '''
    vocab = {element[0]: index for index, element in enumerate(counter.most_common(self.max_size))}
    return vocab

  def fit(self, texts):
    '''
    Metodo que cria os argumentos que serao utilizados nas
    transformacoes posteriores. Esse metodo so deve ser utilizado 
    sobre o conjunto de treino.

    Args:
      texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    '''
    self.len_corpus = len(texts)
    counter = self._count_tokens_in_doc(texts)

    self.vocabulary = self._create_vocab(counter)
    self.idf = self._create_idf(counter)

  def transform(self, texts):
    '''
    Realiza a transformacao de uma lista de tokens para uma
    lista de inteiros com base no vocabulario criado na etapa
    de fit.

    Args:
      texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    
    Return:
      tfidf_texts(torch.tensor): Array contendo os vetores de tokens
        transformados para vetores de inteiros de tamanho fixo.
    '''
    transformed_texts = []

    for i, text in enumerate(texts):
      bow_text = torch.zeros(len(self.vocabulary))
      counter = Counter(text)

      index = []
      values = []
      for key, value in counter.items():
        if key in self.vocabulary.keys():
          index.append(self.vocabulary[key])
          values.append(value)

      bow_text[index] = torch.Tensor(values)

      transformed_texts.append(bow_text * self.idf)

    return torch.vstack(transformed_texts).float()

In [ ]:
## Vocabulario: {'t1': 0, 't2': 1, 't3': 2, 't4': 3}

## Validando o TFIDF
texts = [['t1', 't2', 't3', 't2', 't1'], ['t2', 't1'], ['t4', 't1']]
tfidf = TfIdf()
tfidf.fit(texts)
assert torch.all(tfidf.transform(texts) - torch.Tensor([[0., 2 * np.log(3/2), np.log(3), 0.], [0., np.log(3/2), 0., 0.] , [0., 0., 0., np.log(3)]]) < 0.001)

## Tokenização

Irei aplicar uma tokenização simples, onde irei remover a pontuação do texto e irei dividí-lo por palavras, ou seja, meus tokens serão as palavras que compõe a avaliação do filme.

In [ ]:
from re import findall

def tokenizer(texts):
  tokenized_texts = []
  for text in texts:
    tokens = findall(r'\w+|[^?\-!.,:;"\'/><\s]', text)
    tokenized = [token.lower() for token in tokens]
    tokenized_texts.append(tokenized)
  return tokenized_texts

In [ ]:
tokenized_x_train = tokenizer(x_train)
tokenized_x_valid = tokenizer(x_valid)
tokenized_x_test = tokenizer(x_test)

## Input para a rede neural

No Pytorch, para treinarmos uma rede neural, é de extrema importância criarmos uma estrutura chamada Dataloader, a qual retorna, em cada iteração, um batch do tamanho definido. Para criarmos essa estrutura podemos definir uma outra classe, chamada Dataset, a qual temos que sobrescrever dois métodos: \_\_len\_\_ e \_\_getitem\_\_. Abaixo, podemos ver a definição do Dataset e a criação de uma função que cria os Dataloaders para utilizarmos no treino e teste. 

In [ ]:
from torch.utils.data import Dataset

class VectorizedDataset(Dataset):
    def __init__(self, x, y, vectorizer):
        self.x = x
        self.y = y
        self.vectorizer = vectorizer

    def __len__(self) -> int:
        return len(self.x)

    def __getitem__(self, idx):
        x = self.vectorizer.transform([self.x[idx]])[0]
        y = self.y[idx]
        return x, y

In [ ]:
from torch.utils.data import DataLoader

def create_nn_input(tokenized_text_train, y_train,
                    tokenized_text_valid, y_valid,
                    tokenized_text_test, y_test,
                    vectorizer, batch_size, shuffle):
  
  vectorizer.fit(tokenized_text_train)

  train_dataset = VectorizedDataset(tokenized_text_train, torch.Tensor(y_train).reshape(-1,1), vectorizer)
  valid_dataset = VectorizedDataset(tokenized_text_valid, torch.Tensor(y_valid).reshape(-1,1), vectorizer)
  test_dataset = VectorizedDataset(tokenized_text_test, torch.Tensor(y_test).reshape(-1,1), vectorizer)

  vectorized_texts_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
  vectorized_texts_valid = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
  vectorized_texts_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return vectorized_texts_train, vectorized_texts_valid, vectorized_texts_test, vectorizer

## Definição da Rede Neural

Na célula abaixo defino minha rede neural. Utilizarei 3 camadas escondidas com 128, 64 e 32 neurônios e, na camada de saída, teremos apenas uma saída e a aplicação de uma função sigmoide, que trará a probabilidade de ser positiva a avaliação (se > 0.5, é positiva, caso contrário, negativa).

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_units=128):
        super().__init__()
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(input_size, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 1),
            torch.nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.dense(x)
        return x

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
   print(torch. cuda. get_device_name(dev))
else: 
   dev = "cpu" 
print(dev)
device = torch.device(dev)

Tesla P4
cuda:0


In [ ]:
mlp = MLP(10000)
mlp.to(device)

MLP(
  (dense): Sequential(
    (0): Linear(in_features=10000, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

### Treino da rede neural

Abaixo, temos a definição do loop de treino:

In [ ]:
def train(model, train, valid, criterion, optimizer, filename_save, n_epochs=10):
  
  best_valid_loss = 10e9
  best_epoch = 0

  for i in range(n_epochs):
    accumulated_loss = 0
    model.train()
    for x_train, y_train in train:
      x_train = x_train.to(device)
      y_train = y_train.to(device)
      outputs = model(x_train)
      batch_loss = criterion(outputs, y_train)

      optimizer.zero_grad()
      batch_loss.backward()
      optimizer.step()
      accumulated_loss += batch_loss.item()

    train_loss = accumulated_loss / len(train.dataset)
    
    # Laço de Validação, um a cada época.
    accumulated_loss = 0
    accumulated_accuracy = 0
    model.eval()
    with torch.no_grad():
        for x_valid, y_valid in valid:
            x_valid = x_valid.to(device)
            y_valid = y_valid.to(device)

            # predict da rede
            outputs = model(x_valid)

            # calcula a perda
            batch_loss = criterion(outputs, y_valid)
            preds = outputs > 0.5

            # calcula a acurácia
            batch_accuracy = (preds == y_valid).sum()
            accumulated_loss += batch_loss
            accumulated_accuracy += batch_accuracy

    valid_loss = accumulated_loss / len(valid.dataset)
    valid_acc = accumulated_accuracy / len(valid.dataset)
    print(f'Época: {i:d}/{n_epochs - 1:d} Train Loss: {train_loss:.6f} Valid Loss: {valid_loss:.6f} Valid Acc: {valid_acc:.3f}')

    # Salvando o melhor modelo de acordo com a loss de validação
    if valid_loss < best_valid_loss:
        torch.save(model.state_dict(), filename_save + '.pt')
        best_valid_loss = valid_loss
        best_epoch = i
        print('best model')

  return model

## Loss e otimizador:

Na célula a seguir estou definindo a Loss a ser utilizada (binary cross entropy, já que temos apenas duas classes) e Stochastic gradient descent como otimizador. Manterei os parâmetros para todos os experimentos.

In [ ]:
learningRate = 0.01

# Utilizaremos CrossEntropyLoss como função de perda
criterion = torch.nn.BCELoss()

# Gradiente descendente
optimizer = torch.optim.SGD(mlp.parameters(), lr=learningRate)

## Avaliação no conjunto de teste:

In [ ]:
def predict(model, state_dict, test):
  accumulated_accuracy = 0
  model.load_state_dict(torch.load(state_dict + '.pt'))
  model.eval()
  with torch.no_grad():
      for x_test, y_test in test:
          x_test = x_test.to(device)
          y_test = y_test.to(device)

          # predict da rede
          outputs = model(x_test)

          # calcula a perda
          batch_loss = criterion(outputs, y_test)
          preds = outputs > 0.5

          # calcula a acurácia
          batch_accuracy = (preds == y_test).sum()
          accumulated_accuracy += batch_accuracy

  test_acc = accumulated_accuracy / len(test.dataset)
  test_acc *= 100
  print('*' * 40)
  print(f'Acurácia de {test_acc:.3f} %')
  print('*' * 40)

## Experimentos

### Bow Booleano

In [ ]:
max_size = 3000
batch_size = 64
n_epochs = 10
learningRate = 0.1
save_filename = 'bow_bool'

In [ ]:
mlp_bow_bool = MLP(max_size)
mlp_bow_bool.to(device)

MLP(
  (dense): Sequential(
    (0): Linear(in_features=3000, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [ ]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(mlp_bow_bool.parameters(), lr=learningRate)

In [ ]:
vectorizer = BagOfWords(boolean=True, max_size = max_size)

vectorized_train, vectorized_valid, vectorized_test, _ = create_nn_input(tokenized_x_train, y_train,
                                                                         tokenized_x_valid, y_valid,
                                                                         tokenized_x_test, y_test,
                                                                         vectorizer, batch_size, True)

_ = train(mlp_bow_bool, vectorized_train, vectorized_valid, criterion,
          optimizer, save_filename, n_epochs=n_epochs)

Época: 0/9 Train Loss: 0.010083 Valid Loss: 0.006647 Valid Acc: 0.831
best model
Época: 1/9 Train Loss: 0.005940 Valid Loss: 0.005311 Valid Acc: 0.859
best model
Época: 2/9 Train Loss: 0.004731 Valid Loss: 0.005377 Valid Acc: 0.859
Época: 3/9 Train Loss: 0.004157 Valid Loss: 0.005544 Valid Acc: 0.854
Época: 4/9 Train Loss: 0.003747 Valid Loss: 0.004819 Valid Acc: 0.880
best model
Época: 5/9 Train Loss: 0.003460 Valid Loss: 0.005128 Valid Acc: 0.877
Época: 6/9 Train Loss: 0.003036 Valid Loss: 0.004947 Valid Acc: 0.874
Época: 7/9 Train Loss: 0.002731 Valid Loss: 0.005468 Valid Acc: 0.871
Época: 8/9 Train Loss: 0.002243 Valid Loss: 0.005766 Valid Acc: 0.869
Época: 9/9 Train Loss: 0.002208 Valid Loss: 0.005378 Valid Acc: 0.867


In [ ]:
predict(mlp_bow_bool, save_filename, vectorized_test)

****************************************
Acurácia de 87.428 %
****************************************


### BoW frequência

In [ ]:
max_size = 3000
batch_size = 64
n_epochs = 10
learningRate = 0.1
save_filename = 'bow_freq'

In [ ]:
mlp_bow_freq = MLP(max_size)
mlp_bow_freq.to(device)

MLP(
  (dense): Sequential(
    (0): Linear(in_features=3000, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [ ]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(mlp_bow_freq.parameters(), lr=learningRate)

In [ ]:
vectorizer = BagOfWords(max_size=max_size)

vectorized_train, vectorized_valid, vectorized_test, _ = create_nn_input(tokenized_x_train, y_train,
                                                                         tokenized_x_valid, y_valid,
                                                                         tokenized_x_test, y_test,
                                                                         vectorizer, batch_size, True)

_ = train(mlp_bow_freq, vectorized_train, vectorized_valid, criterion,
          optimizer, save_filename, n_epochs=n_epochs)

Época: 0/9 Train Loss: 0.010010 Valid Loss: 0.010389 Valid Acc: 0.648
best model
Época: 1/9 Train Loss: 0.008421 Valid Loss: 0.007636 Valid Acc: 0.774
best model
Época: 2/9 Train Loss: 0.007375 Valid Loss: 0.007630 Valid Acc: 0.748
best model
Época: 3/9 Train Loss: 0.006855 Valid Loss: 0.006338 Valid Acc: 0.835
best model
Época: 4/9 Train Loss: 0.006291 Valid Loss: 0.005637 Valid Acc: 0.854
best model
Época: 5/9 Train Loss: 0.005811 Valid Loss: 0.005923 Valid Acc: 0.840
Época: 6/9 Train Loss: 0.005661 Valid Loss: 0.006150 Valid Acc: 0.821
Época: 7/9 Train Loss: 0.005237 Valid Loss: 0.005180 Valid Acc: 0.867
best model
Época: 8/9 Train Loss: 0.005247 Valid Loss: 0.005864 Valid Acc: 0.834
Época: 9/9 Train Loss: 0.004928 Valid Loss: 0.005831 Valid Acc: 0.859


In [ ]:
predict(mlp_bow_freq, save_filename, vectorized_test)

****************************************
Acurácia de 86.384 %
****************************************


### TF-IDF

In [ ]:
max_size = 3000
batch_size = 64
n_epochs = 10
learningRate = 0.01
save_filename = 'tfidf'

In [ ]:
mlp_tfidf = MLP(max_size)
mlp_tfidf.to(device)

MLP(
  (dense): Sequential(
    (0): Linear(in_features=3000, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [ ]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(mlp_tfidf.parameters(), lr=learningRate)

In [ ]:
vectorizer = TfIdf(max_size=max_size)

vectorized_train, vectorized_valid, vectorized_test, _ = create_nn_input(tokenized_x_train, y_train,
                                                                         tokenized_x_valid, y_valid,
                                                                         tokenized_x_test, y_test,
                                                                         vectorizer, batch_size, True)

_ = train(mlp_tfidf, vectorized_train, vectorized_valid, criterion,
          optimizer, save_filename, n_epochs=n_epochs)

Época: 0/9 Train Loss: 0.010833 Valid Loss: 0.010912 Valid Acc: 0.596
best model
Época: 1/9 Train Loss: 0.010750 Valid Loss: 0.010779 Valid Acc: 0.681
best model
Época: 2/9 Train Loss: 0.010412 Valid Loss: 0.010074 Valid Acc: 0.781
best model
Época: 3/9 Train Loss: 0.008645 Valid Loss: 0.007109 Valid Acc: 0.847
best model
Época: 4/9 Train Loss: 0.005563 Valid Loss: 0.005180 Valid Acc: 0.875
best model
Época: 5/9 Train Loss: 0.004282 Valid Loss: 0.005118 Valid Acc: 0.874
best model
Época: 6/9 Train Loss: 0.003744 Valid Loss: 0.004959 Valid Acc: 0.877
best model
Época: 7/9 Train Loss: 0.003407 Valid Loss: 0.005075 Valid Acc: 0.880
Época: 8/9 Train Loss: 0.003204 Valid Loss: 0.005311 Valid Acc: 0.878
Época: 9/9 Train Loss: 0.002964 Valid Loss: 0.005546 Valid Acc: 0.873


In [ ]:
predict(mlp_tfidf, save_filename, vectorized_test)

****************************************
Acurácia de 87.912 %
****************************************
